In [1]:
import pandas as pd 
import numpy as np

file1 = 'c:/users/wston/documents/python/pybuild/TSLA_x.csv'

jump = pd.read_csv(file1)
datetime_series = pd.DatetimeIndex(jump['Date'])
datetime_series
jump = jump.set_index(datetime_series)
jump
# jump.to_csv('c:/users/wston/documents/stock_data.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2010-06-30,2010-06-30,1.719333,2.028000,1.553333,1.588667,1.588667,257806500
2010-07-01,2010-07-01,1.666667,1.728000,1.351333,1.464000,1.464000,123282000
2010-07-02,2010-07-02,1.533333,1.540000,1.247333,1.280000,1.280000,77097000
2010-07-06,2010-07-06,1.333333,1.333333,1.055333,1.074000,1.074000,103003500
2010-07-07,2010-07-07,1.093333,1.108667,0.998667,1.053333,1.053333,103825500
...,...,...,...,...,...,...,...
2019-12-13,2019-12-13,24.070000,24.347334,23.642668,23.892668,23.892668,98563500
2019-12-16,2019-12-16,24.170000,25.573999,24.166668,25.433332,25.433332,272613000
2019-12-17,2019-12-17,25.266001,25.700001,25.059999,25.266001,25.266001,127452000


In [2]:
num_obs = jump[['Close']].count()
num_obs


def add_memory(s,n_days=50,mem_strength=0.1):
    ''' adds autoregressive behavior to series of data'''
    add_ewm = lambda x: (1-mem_strength)*x + mem_strength*x.ewm(n_days).mean()
    out = s.groupby(level='Date').apply(add_ewm)
    return out

# generate feature data
f01 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f01 = add_memory(f01,10,0.1)
f02 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f02 = add_memory(f02,10,0.1)
f03 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f03 = add_memory(f03,10,0.1)
f04 = pd.Series(np.random.randn(2386),index=jump[['Close']].index)
f04 = f04 # no memory

features = pd.concat([f01,f02,f03,f04],axis=1)
features.to_csv('c:/users/wston/documents/jump_data.csv')

outcome =   f01 * np.linspace(0.5,1.5,2386) + \
            f02 * np.linspace(1.5,0.5,2386) + \
            f03 * pd.Series(np.sin(2*np.pi*np.linspace(0,1,2386)*2)+1,index=f03.index) + \
            f04 + \
            np.random.randn(2386) * 3 
outcome.name = 'outcome'
outcome

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/1919133032.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  out = s.groupby(level='Date').apply(add_ewm)
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/1919133032.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  out = s.groupby(level='Date

Date
2010-06-30    2.242174
2010-07-01   -2.557357
2010-07-02    7.544811
2010-07-06    5.603336
2010-07-07    3.185056
                ...   
2019-12-13    6.544745
2019-12-16    1.957285
2019-12-17   -3.917396
2019-12-18   -1.334963
2019-12-19    6.048675
Name: outcome, Length: 2386, dtype: float64

In [3]:
from sklearn.linear_model import LinearRegression
from itertools import chain

recalc_dates = features.resample('D',level='Date').mean().values[:-1]
# print('init', str(recalc_dates))

flat = list(chain.from_iterable(recalc_dates))
print(flat)
recalc_dates = pd.Series(flat)
models = pd.Series(index=recalc_dates)
for date in recalc_dates:
    X_train = features.iloc[0:2386]
    y_train = outcome.iloc[0:2386]
    model = LinearRegression()
    model.fit(X_train,y_train)
    models.loc[date] = model

## predict values walk-forward (all predictions out of sample)
begin_dates = models.index
end_dates = models.index[1:].append(pd.to_datetime(['2099-12-31']))

predictions = pd.Series(index=features.index)

for i,model in enumerate(models): #loop thru each models object in collection
    X = features.iloc[0:2386]
    p = pd.Series(model.predict(X),index=X.index)
    predictions.loc[X.index] = p

c:\Users\wston\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[-0.8535740834588326, 1.3962219801046434, -0.3285549354686423, -0.8346180932913572, -0.681889672755231, 0.35959491923545284, 0.034308749827383876, 0.6972295867798979, 0.5502944752465317, -0.3720941486418774, 1.0738978559113816, -0.5342811811972078, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 1.63624069278965, 0.6231265235196622, -0.3411653047524725, 1.8713487566384996, 1.224278284522184, 0.11674063143217911, -0.03890165949757848, -0.370148698978572, -0.06328038734601295, 0.2876715706281972, -0.7413133416653258, 2.3762303244070053, -1.4721838656293205, -0.4278061547823813, -0.4769016060214965, 0.2673997821173135, nan, nan, nan, nan, nan, nan, nan, nan, 0.6024584639945125, -0.30272619114910415, -0.5895823850511297, -0.5272496582938632, -0.141198735956412, 0.8117314137708956, -0.591924614829592, -0.23219060622341944, 1.260518560662509, -0.6823131051872355, 2.050481150919608, -0.9487895645771791, -0.09482116155207557, 1.1188300158856115, -1.1604275007799922, 0.0594882480848

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2519312849.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  models = pd.Series(index=recalc_dates)
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2519312849.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  predictions = pd.Series(index=features.index)


In [4]:
import sklearn.metrics as metrics

# make sure we have 1-for-1 mapping between pred and true
common_idx = outcome.dropna().index.intersection(predictions.dropna().index)
y_true = outcome[common_idx]
y_true.name = 'y_true'
y_pred = predictions[common_idx]
y_pred.name = 'y_pred'

standard_metrics = pd.Series()

standard_metrics.loc['explained variance'] = metrics.explained_variance_score(y_true, y_pred)
standard_metrics.loc['MAE'] = metrics.mean_absolute_error(y_true, y_pred)
standard_metrics.loc['MSE'] = metrics.mean_squared_error(y_true, y_pred)
standard_metrics.loc['MedAE'] = metrics.median_absolute_error(y_true, y_pred)
standard_metrics.loc['RSQ'] = metrics.r2_score(y_true, y_pred)

print(standard_metrics)

explained variance    0.305377
MAE                   2.456356
MSE                   9.517904
MedAE                 2.032636
RSQ                   0.305377
dtype: float64


C:\Users\wston\AppData\Local\Temp/ipykernel_9436/164972062.py:10: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  standard_metrics = pd.Series()


In [5]:
print(pd.concat([y_pred,y_true],axis=1).tail())

              y_pred    y_true
Date                          
2019-12-13  2.328127  6.544745
2019-12-16  1.698282  1.957285
2019-12-17 -0.643072 -3.917396
2019-12-18 -0.166246 -1.334963
2019-12-19 -0.153091  6.048675


In [6]:
def make_df(y_pred,y_true):
    y_pred.name = 'y_pred'
    y_true.name = 'y_true'
    
    df = pd.concat([y_pred,y_true],axis=1)

    df['sign_pred'] = df.y_pred.apply(np.sign)
    df['sign_true'] = df.y_true.apply(np.sign)
    df['is_correct'] = 0
    df.loc[df.sign_pred * df.sign_true > 0 ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
    df['is_incorrect'] = 0
    df.loc[df.sign_pred * df.sign_true < 0,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
    df['is_predicted'] = df.is_correct + df.is_incorrect
    df['result'] = df.sign_pred * df.y_true 
    return df

df = make_df(y_pred,y_true)
# print(df.dropna().tail())
df

,y_pred,y_true,sign_pred,sign_true,is_correct,is_incorrect,is_predicted,result
Date,,,,,,,,
2010-06-30,-0.612623,2.242174,-1.0,1.0,0,1,1,-2.242174
2010-07-01,0.502470,-2.557357,1.0,-1.0,0,1,1,-2.557357
2010-07-02,0.703712,7.544811,1.0,1.0,1,0,1,7.544811
2010-07-06,3.930563,5.603336,1.0,1.0,1,0,1,5.603336
2010-07-07,0.939956,3.185056,1.0,1.0,1,0,1,3.185056
...,...,...,...,...,...,...,...,...
2019-12-13,2.328127,6.544745,1.0,1.0,1,0,1,6.544745
2019-12-16,1.698282,1.957285,1.0,1.0,1,0,1,1.957285
2019-12-17,-0.643072,-3.917396,-1.0,-1.0,1,0,1,3.917396


In [7]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    
    return scorecard    

calc_scorecard(df)

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/8326907.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()


accuracy    68.273261
edge         1.635173
noise        2.300862
dtype: float64

In [8]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    return scorecard    

calc_scorecard(df)

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/1472535695.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()


accuracy                  68.273261
edge                       1.635173
noise                      2.300862
y_true_chg                 2.915330
y_pred_chg                 1.627482
prediction_calibration     0.558250
capture_ratio             56.088764
dtype: float64

In [9]:
def calc_scorecard(df):
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['edge'] = df.result.mean()
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()

    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    scorecard.loc['prediction_calibration'] = scorecard.loc['y_pred_chg']/scorecard.loc['y_true_chg']
    scorecard.loc['capture_ratio'] = scorecard.loc['edge']/scorecard.loc['y_true_chg']*100

    # metrics for a subset of predictions
    scorecard.loc['edge_long'] = df[df.sign_pred == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_short'] = df[df.sign_pred == -1].result.mean()  - df.y_true.mean()

    scorecard.loc['edge_win'] = df[df.is_correct == 1].result.mean()  - df.y_true.mean()
    scorecard.loc['edge_lose'] = df[df.is_incorrect == 1].result.mean()  - df.y_true.mean()

    return scorecard    

calc_scorecard(df)

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()


accuracy                  68.273261
edge                       1.635173
noise                      2.300862
y_true_chg                 2.915330
y_pred_chg                 1.627482
prediction_calibration     0.558250
capture_ratio             56.088764
edge_long                  1.560705
edge_short                 1.407453
edge_win                   3.184812
edge_lose                 -2.165228
dtype: float64

In [10]:
def scorecard_by_year(df):
    df['year'] = df.index.get_level_values('Date').year
    return df.groupby('year').apply(calc_scorecard).T

print(scorecard_by_year(df))

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = 

year                         2010       2011       2012       2013       2014  \
accuracy                66.666667  69.444444  72.400000  64.285714  65.079365   
edge                     1.708998   1.815412   2.026567   1.125018   1.556669   
noise                    2.363001   2.293085   2.209771   2.170241   2.469415   
y_true_chg               2.931885   3.090693   3.159454   2.634615   2.978685   
y_pred_chg               1.708757   1.616643   1.604749   1.468861   1.692196   
prediction_calibration   0.582819   0.523068   0.507920   0.557524   0.568102   
capture_ratio           58.290085  58.738002  64.142946  42.701414  52.260287   
edge_long                1.790097   1.798275   1.611512   1.164381   1.618808   
edge_short               1.052028   1.105139   1.690073   1.734057   0.842048   
edge_win                 3.178423   3.171442   3.197561   3.261924   3.145909   
edge_lose               -2.136570  -2.447779  -2.436276  -1.775672  -2.374639   

year                       

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()


In [13]:
def scorecard_by_symbol(df):
    return df.groupby(level='Date').apply(calc_scorecard).T

print(scorecard_by_symbol(df))

C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = 

Date                    2010-06-30  2010-07-01  2010-07-02  2010-07-06  \
accuracy                  0.000000    0.000000  100.000000  100.000000   
edge                     -2.242174   -2.557357    7.544811    5.603336   
noise                          NaN         NaN         NaN         NaN   
y_true_chg                2.242174    2.557357    7.544811    5.603336   
y_pred_chg                0.612623    0.502470    0.703712    3.930563   
prediction_calibration    0.273227    0.196480    0.093271    0.701468   
capture_ratio          -100.000000 -100.000000  100.000000  100.000000   
edge_long                      NaN    0.000000    0.000000    0.000000   
edge_short               -4.484347         NaN         NaN         NaN   
edge_win                       NaN         NaN    0.000000    0.000000   
edge_lose                -4.484347    0.000000         NaN         NaN   

Date                    2010-07-07  2010-07-08  2010-07-09  2010-07-12  \
accuracy                100.000000  1

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor

X_train,X_test,y_train,y_test = train_test_split(features,outcome,test_size=0.20,shuffle=False)

# linear regression
model1 = LinearRegression().fit(X_train,y_train)
model1_train = pd.Series(model1.predict(X_train),index=X_train.index)
model1_test = pd.Series(model1.predict(X_test),index=X_test.index)

model2 = RandomForestRegressor().fit(X_train,y_train)
model2_train = pd.Series(model2.predict(X_train),index=X_train.index)
model2_test = pd.Series(model2.predict(X_test),index=X_test.index)

# create dataframes for each 
model1_train_df = make_df(model1_train,y_train)
model1_test_df = make_df(model1_test,y_test)
model2_train_df = make_df(model2_train,y_train)
model2_test_df = make_df(model2_test,y_test)

s1 = calc_scorecard(model1_train_df)
s1.name = 'model1_train'
s2 = calc_scorecard(model1_test_df)
s2.name = 'model1_test'
s3 = calc_scorecard(model2_train_df)
s3.name = 'model2_train'
s4 = calc_scorecard(model2_test_df)
s4.name = 'model2_test'

print(pd.concat([s1,s2,s3,s4],axis=1))

                        model1_train  model1_test  model2_train  model2_test
accuracy                   69.339623    65.062762     91.352201    62.133891
edge                        1.762990     1.208632      2.859010     1.044569
noise                       2.451991     2.399001      3.217855     2.466705
y_true_chg                  2.963876     2.721553      2.963876     2.721553
y_pred_chg                  1.725783     1.727178      2.289583     1.759678
prediction_calibration      0.582272     0.634630      0.772496     0.646571
capture_ratio              59.482559    44.409660     96.461853    38.381356
edge_long                   1.689907     1.051569      2.815566     0.947330
edge_short                  1.540416     1.046335      2.609732     0.811161
edge_win                    3.263638     2.860962      3.042202     2.871309
edge_lose                  -2.103220    -2.324539     -0.751169    -2.373701


C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = pd.Series()
C:\Users\wston\AppData\Local\Temp/ipykernel_9436/2304184531.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  scorecard = 